In [18]:
import numpy as np
import pandas as pd
import sys
from openai import OpenAI
import os
import cv2
import re
import glob
from pathlib import Path
import matplotlib.pyplot as plt
import base64
from tqdm import tqdm
from dotenv import load_dotenv
import re
from groq import Groq

In [19]:
df = pd.read_csv('Data/New230SHEETS.csv')
df

,id,image_name,ocr_txt
0,1,Meme_Data_1.jpg,COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK...
1,2,Meme_Data_2.jpg,ayat suci dalam bio bukan jaminan kalo\n dia o...
2,3,Meme_Data_3.jpg,Rare\n Medium Rare\n Medium\n Medium Well\n We...
3,4,Meme_Data_4.jpg,KITA MAH ORANG SUSAH\n INWE\n SUSAH DILAWAN MA...
4,5,Meme_Data_5.jpg,KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FAN...
...,...,...,...
225,226,Meme_Data_226.jpg,": ""Kamu dari jawa ya!"" Iyaa, kamu jawa juga?\n..."
226,227,Meme_Data_227.jpg,"Pas kecil liat orang dewasa pacaran, pas sudah..."
227,228,Meme_Data_228.jpg,SINGAPORE AIRLINES\n 1. Untung 47 Trilliun 2. ...
228,229,Meme_Data_229.jpg,Yudi\n \n P\n \n Bls\n \n Besok pake baju apa\...


In [20]:
# Check for missing values in the 'image_name' column
missing_image_names = df['image_name'].isna()

# Show rows where 'image_name' is missing
df_missing = df[missing_image_names]
print(df_missing)


Empty DataFrame
Columns: [id, image_name, ocr_txt]
Index: []


In [21]:
# Drop rows where 'image_name' is missing
df = df.dropna(subset=['ocr_txt'])

# Optionally, reset the index if you want a clean index after dropping
df = df.reset_index(drop=True)

In [22]:
# Load prompt template
with open('Prompt/interpreter.md', 'r', encoding='utf-8') as f:
    prompt_template = f.read()

In [23]:
# Test with first meme
test_row = df.iloc[0]
image_text = test_row['ocr_txt']
image_name = test_row['image_name']

# Replace placeholders
final_prompt = prompt_template.replace('{meme_text}', str(image_text))
final_prompt = final_prompt.replace('{image_path}', image_name)

# Show the final prompt
print("FINAL PROMPT THAT WILL BE SENT TO OPENAI:")
print("="*60)
print(final_prompt)
print("="*60)

FINAL PROMPT THAT WILL BE SENT TO OPENAI:
# Role
You are an expert in interpreting meme since you are a master of analyzing the hidden message behind meme in Indonesian language

# Task
Your task is to generate a hidden message or the context of the meme

## Things Need to be Considered
- There are 2 inputs which are meme image and meme text. Please, refer to those input for interpret meme
- All of the memes are in Indonesian language. Therefore, please interpret meme in Indonesian language
- The connection between meme image and meme text can be metaporical. Therefore, analyze both of those input carefully (i.e., a picture of a grumpy man is represent bad stuff and dislike)

## Example

### Example 1
Input Image = meme_data_999.jpg
Input Text = Sudah Kubilang jangan lawan King Indonesia
Output = Tim sepak bola Indonesia adalah yang terbaik saat ini dan susah untuk dikalahkan. Oleh karena itu jangan coba-coba melawan tim sepak bola Indonesia

## Query
Input Image = Meme_Data_1.jpg
Inpu

In [24]:
def encode_image_to_base64(image_path):
    """Encode image to base64"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [25]:
# Load environment variables
load_dotenv()

# Initialize OpenAI client with API key from environment
client = Groq(
    api_key=os.getenv('GROQ_API_KEY')
)

# Verify API key is loaded
if not os.getenv('GROQ_API_KEY'):
    raise ValueError("GROQ_API_KEY not found in environment variables. Please create a .env file with your API key.")

def interpret_meme(row):
    """Interpret meme using OpenAI Vision API"""
    # Get image path and text from your CSV structure
    image_path = f"Data/Filtered 230/{row['image_name']}"
    image_text = row['ocr_txt']
    
    # Encode image
    base64_image = encode_image_to_base64(image_path)
    
    # Create prompt by replacing placeholders
    prompt = prompt_template.replace('{meme_text}', str(image_text))
    prompt = prompt.replace('{image_path}', row['image_name'])
    
    try:
        response = client.chat.completions.create(
            model="meta-llama/llama-4-scout-17b-16e-instruct",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}",
                                "detail": "high"
                            }
                        }
                    ]
                }
            ],
            max_tokens=2048,
            temperature=0
        )

        result = response.choices[0].message.content.strip()
        return result
    except Exception as e:
        print("Error:", e)
        return None

In [29]:
sample_row = df.iloc[0]  # Use the first row for testing
output = interpret_meme(sample_row)
print(output)

Berdasarkan gambar dan teks meme yang disediakan, dapat diartikan bahwa:

Laki-laki zaman sekarang ingin memiliki pacar yang cantik, namun ketika pacarnya meminta uang untuk perawatan di salon, laki-laki tersebut menyebut pacarnya sebagai "matre" (istilah yang mungkin digunakan untuk menggambarkan seseorang yang dianggap materialistis). 

Teks pada gambar juga menambahkan pertanyaan, "Emang lo kira perawatan di salon pake BPJS??" yang artinya "Apakah kamu berpikir bahwa perawatan di salon bisa menggunakan BPJS?" 

BPJS (Badan Penyelenggara Jaminan Sosial) adalah program jaminan sosial di Indonesia yang biasanya digunakan untuk biaya kesehatan dan perawatan medis, bukan untuk perawatan kecantikan seperti salon.

Maka, konteks atau pesan tersembunyi dari meme ini adalah kritik atau ejekan terhadap laki-laki yang ingin memiliki pacar cantik tapi tidak mau mengeluarkan uang untuk kebutuhan pacarnya, terutama yang berkaitan dengan perawatan kecantikan. Meme ini juga menyiratkan bahwa bebera

In [30]:
# Process all memes
results = []
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing memes"):
    result = interpret_meme(row)
    results.append({
        'id': row['id'],
        'image_name': row['image_name'],
        'ocr_txt': row['ocr_txt'],
        'full_response': result
    })
    
    # Optional: Add delay to respect rate limits
    import time
    time.sleep(1)

results

Processing memes: 100%|██████████| 230/230 [10:52<00:00,  2.84s/it]


[{'id': 1,
  'image_name': 'Meme_Data_1.jpg',
  'ocr_txt': 'COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK GILIRAN DIMINTAIN DUIT SALON, BILANGNYA MATRE EMANG LO KIRA PERAWATAN DI SALON PAKE BPJS??',
  'full_response': 'Berdasarkan gambar dan teks meme yang disediakan, pesan tersembunyi di balik meme ini adalah kritik sosial terhadap cowok (laki-laki) yang menginginkan pacar cantik namun tidak mau mengeluarkan uang untuk kebutuhan pacarnya, seperti biaya perawatan di salon. Gambar tiga wanita dengan pose yang sama, masing-masing dengan hiasan yang berbeda di kepala mereka, mungkin melambangkan standar kecantikan yang tinggi yang diinginkan oleh laki-laki.\n\nTeks "COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK GILIRAN DIMINTAIN DUIT SALON, BILANGNYA MATRE EMANG LO KIRA PERAWATAN DI SALON PAKE BPJS??" secara langsung menyampaikan keluhan bahwa laki-laki zaman sekarang ingin memiliki pacar yang cantik, tapi ketika pacarnya meminta uang untuk perawatan di salon, mereka malah menyebut paca

In [41]:
# Create results DataFrame
results_df = pd.DataFrame(results)
results_df

,id,image_name,ocr_txt,full_response
0,1,Meme_Data_1.jpg,COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK...,Berdasarkan gambar dan teks meme yang disediak...
1,2,Meme_Data_2.jpg,ayat suci dalam bio bukan jaminan kalo\n dia o...,Output = \nMeme ini ingin menyampaikan bahwa m...
2,3,Meme_Data_3.jpg,Rare\n Medium Rare\n Medium\n Medium Well\n We...,Meme ini menggunakan gambar potongan daging de...
3,4,Meme_Data_4.jpg,KITA MAH ORANG SUSAH\n INWE\n SUSAH DILAWAN MA...,"Kita ini orang yang tangguh dan kuat, sehingga..."
4,5,Meme_Data_5.jpg,KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FAN...,Output = Meme ini menyampaikan kritik terhadap...
...,...,...,...,...
225,226,Meme_Data_226.jpg,": ""Kamu dari jawa ya!"" Iyaa, kamu jawa juga?\n...",Orang Jawa sering kali merujuk pada penduduk y...
226,227,Meme_Data_227.jpg,"Pas kecil liat orang dewasa pacaran, pas sudah...",Konteks dari meme ini adalah tentang perasaan ...
227,228,Meme_Data_228.jpg,SINGAPORE AIRLINES\n 1. Untung 47 Trilliun 2. ...,Meme ini membandingkan kinerja antara Singapor...
228,229,Meme_Data_229.jpg,Yudi\n \n P\n \n Bls\n \n Besok pake baju apa\...,Output = \nMeme ini menggambarkan seseorang ya...


In [42]:
def extract_combined_robust(response):
    # Try to extract after 'output =' or 'output :'
    match_output = re.search(r'output\s*[:=]\s*([\s\S]*)', response, re.IGNORECASE)
    if match_output:
        return match_output.group(1).strip()
    # Try to extract after 'Pesan tersembunyi dari meme ini adalah:'
    match_pesan = re.search(r'Pesan tersembunyi dari meme ini adalah:\s*([\s\S]*)', response, re.IGNORECASE)
    if match_pesan:
        return match_pesan.group(1).strip()
    # Fallback to original response
    return response.strip()

results_df['output_extracted'] = results_df['full_response'].apply(lambda x: extract_combined_robust(str(x)))

In [43]:
# Example usage:
response = results_df['output_extracted'][228]
print(extract_combined_robust(response))

Meme ini menggambarkan seseorang yang tidak memakai seragam pramuka seperti orang lain. Hal ini mengacu pada seseorang yang tidak mengikuti aturan yang berlaku. Dalam caption "day 1 nge frank yudi" dapat diartikan bahwa Yudi tidak memakai seragam pramuka di hari pertama dan ini merupakan awal dari ketidakpatuhannya.


In [44]:
results_df

,id,image_name,ocr_txt,full_response,output_extracted
0,1,Meme_Data_1.jpg,COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK...,Berdasarkan gambar dan teks meme yang disediak...,Berdasarkan gambar dan teks meme yang disediak...
1,2,Meme_Data_2.jpg,ayat suci dalam bio bukan jaminan kalo\n dia o...,Output = \nMeme ini ingin menyampaikan bahwa m...,Meme ini ingin menyampaikan bahwa memiliki aya...
2,3,Meme_Data_3.jpg,Rare\n Medium Rare\n Medium\n Medium Well\n We...,Meme ini menggunakan gambar potongan daging de...,Meme ini menggunakan gambar potongan daging de...
3,4,Meme_Data_4.jpg,KITA MAH ORANG SUSAH\n INWE\n SUSAH DILAWAN MA...,"Kita ini orang yang tangguh dan kuat, sehingga...","Kita ini orang yang tangguh dan kuat, sehingga..."
4,5,Meme_Data_5.jpg,KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FAN...,Output = Meme ini menyampaikan kritik terhadap...,Meme ini menyampaikan kritik terhadap pemiliha...
...,...,...,...,...,...
225,226,Meme_Data_226.jpg,": ""Kamu dari jawa ya!"" Iyaa, kamu jawa juga?\n...",Orang Jawa sering kali merujuk pada penduduk y...,Meme ini menyindir bahwa orang Jawa adalah ora...
226,227,Meme_Data_227.jpg,"Pas kecil liat orang dewasa pacaran, pas sudah...",Konteks dari meme ini adalah tentang perasaan ...,Konteks dari meme ini adalah tentang perasaan ...
227,228,Meme_Data_228.jpg,SINGAPORE AIRLINES\n 1. Untung 47 Trilliun 2. ...,Meme ini membandingkan kinerja antara Singapor...,Meme ini membandingkan kinerja antara Singapor...
228,229,Meme_Data_229.jpg,Yudi\n \n P\n \n Bls\n \n Besok pake baju apa\...,Output = \nMeme ini menggambarkan seseorang ya...,Meme ini menggambarkan seseorang yang tidak me...


In [45]:
# Save results
results_df.to_csv('meta_maverick_results.csv', index=False)